In [38]:
def gcd(x, y):
   while(y):
       x, y = y, x % y
        
   return x
    
def lcm(x, y):
   return (x * y) // gcd(x, y)

In [40]:
def get_l(p, q):
    return lcm(p - 1, q - 1)

In [81]:
def get_e(l):
    for e in range(2, l):
        if math.gcd(e, l) == 1:
            return e
        else:
            e += 1
            continue
        return 0

In [82]:
def get_d(l, e):
    for d in range(2, l):
        if e * d % l == 1:
            return d
        else:
            d += 1
            continue
        return 0

        

In [83]:
def calc(p, q):
    rsa = dict()
    rsa['p'] = p
    rsa['q'] = q
    rsa['n'] = p * q
    
    rsa['l'] = get_l(p, q)
    l = rsa['l']
    e = get_e(l)
    if e == 0:
        print("can not get e")
        return rsa
    rsa['e'] = e
    
    d = get_d(l, e)
    if d == 0:
        print("can not get d")
        return rsa
    rsa['d'] = d

    return rsa

In [84]:
def verify_rsa(rsa, m_list):
    for m in m_list:
        encrypt = m ** rsa['e'] % rsa['n']
        decrypt = encrypt ** rsa['d'] % rsa['n']
        print("m: {}, encrypt: {}, decrypt: {}, {}".format(m, encrypt, decrypt, m == decrypt))

In [85]:
def get_prime():
    primes = []   # primes generated so far
    last = 1      # last number tried
    while True:
        last += 1
        for p in primes:
            if last % p == 0:
                break
        else:
            primes.append(last)
            yield last



In [86]:
prime = get_prime()

In [87]:
next(prime)

2

In [88]:
def genrsa():
    prime_generator = get_prime()
    for i in (1, 3):
        next(prime_generator)
    p = next(prime_generator)
    for i in range(1, 1000):
        q = next(prime_generator)
        rsa = calc(p, q)
        if rsa.get('e') is None or rsa.get('d') is None:
            continue
        return rsa            

In [89]:
genrsa()

{'p': 5, 'q': 7, 'n': 35, 'l': 12, 'e': 5, 'd': 5}

In [90]:
def main():
    rsa = genrsa()
    verify_rsa(rsa, (1, 3, 6, 11))

In [92]:
main()

m: 1, encrypt: 1, decrypt: 1, True
m: 3, encrypt: 33, decrypt: 3, True
m: 6, encrypt: 6, decrypt: 6, True
m: 11, encrypt: 16, decrypt: 11, True
